
# Deploying a Titanic Survival Prediction Model with Docker and Flask

## Objective
To deploy a machine learning model that predicts the survival rate of passengers on the Titanic using Flask and Docker. The guide covers the end-to-end process, from model building to API testing.

### Technologies Used
- Python
- Flask
- scikit-learn (for machine learning model)
- Docker
- cURL (for API testing)

---


### Preliminary: Setting up a Virtual Environment

Before beginning, it's a good idea to create a Python virtual environment to manage dependencies.

1. Open your terminal and navigate to the project folder.
2. Run the following commands to create and activate a virtual environment:

   **For Windows:**
   ```
   python -m venv venv
   .\venv\Scripts\activate
   pip install Flask scikit-learn pandas
   ```
   
   **For Windows: conda**
   ```
    conda create --name myenv
    conda activate myenv
    conda install flask scikit-learn pandas
   ```

   **For macOS/Linux:**
   ```
   python3 -m venv venv
   source venv/bin/activate
   pip install Flask scikit-learn pandas
   ```

---

### Part 1: Data Loading, Model Fitting, and Saving the Model

**train_model.py**
```python
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load data
df = pd.read_csv('titanic.csv')

# Prepare data
X = df[['Pclass', 'Age', 'SibSp', 'Parch']]
y = df['Survived']
X = X.fillna(0)

# Train model
clf = LogisticRegression(random_state=42,class_weight='balanced',n_jobs=-1)
clf.fit(X, y)

# Save model
joblib.dump(clf, 'model.pkl')
```

---

### Part 2: Flask API for Model Deployment

**app.py**
```python
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)

# Load model
clf = joblib.load('model.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        X = [[data['Pclass'], data['Age'], data['SibSp'], data['Parch']]]
        y_pred = clf.predict(X)
        return jsonify({'prediction': y_pred[0]})
    except Exception as e:
        return jsonify({'error': str(e)})
}

if __name__ == '__main__':
    app.run(host='0.0.0.0')
```

---

### Part 3: Create Requirements File

Create a file named `requirements.txt` and list down all your Python dependencies.

**requirements.txt**
```
Flask==x.x.x
scikit-learn==x.x.x
joblib==x.x.x
pandas==x.x.x
```

where x.x.x is the version that you install

---

### Part 4: Dockerization

**Dockerfile**
```Dockerfile
FROM python:3.10-slim

# Install dependencies
COPY ./requirements.txt /app/requirements.txt
RUN pip install --no-cache-dir -r /app/requirements.txt

# Copy files
COPY ./app.py /app/app.py
COPY ./model.pkl /app/model.pkl

# Run the application
CMD ["python", "/app/app.py"]
```

---

### Part 5: Build and Run Docker Container

1. Build Docker image: 
    ```
    docker build -t titanic-flask-app .
    ```
  
2. Run Docker container:
    ```
    docker run -p 5000:5000 titanic-flask-app
    ```

---

### Part 6: Test with cURL

You can test the API by running this cURL command:

```bash
curl --request POST \
  --url http://localhost:5000/predict \
  --header 'Content-Type: application/json' \
  --data '{
    "Pclass": 3,
    "Age": 34,
    "SibSp": 0,
    "Parch": 0
  }'
```
